In [1]:
import looker_sdk # You'll need to pip install this
import json

# Soft-deleting unused content (moving them to the trash folder)

## 1) Creating a Look that returns unused content

On larger instances, many Looks and Dashboards will stop being viewed after a while. To do this, we're going to create a look that returns all of the unused content, as defined by us (or better yet, your business users). Here's how to do that:

- If you have access to 'System Activity' (your admin will have to give you permission), click the Explore tab, then scroll to the bottom, and select the "Content Usage" explore
- In this explore, there's an 'Inactive Content' quick start that will get you well on your way to returning unused content
- I recommend adding the following filters, if they aren't there already:
  - Content ID doesn't contain `block`
  - Content Type is equal to `look, dashboard`
  - Look Is Used on Dashboard is `No`
  - Look Public is `No`
  - Look Moved to Trash is `No`
  - Dashboard Moved to Trash is `No`
  - Content Usage Favorites Total is equal to `0`
  - Content Usage Schedule Total is equal to `0`
  - Content Usage API Total is equal to `0`
- And some bonus filters that may be helpful depending on your use case
  - Content ID doesn't contain `block` - some blocks don't have ID numbers, and probably shouldn't be deleted
  - Content Title doesn't contain `annual, quarterly, Annual, Yearly, etc...` - Any terms that may indicate infrequently viewed, but important content

## 2) Running the Look via the API

Next thing to do is return all the data from the look we just created, you'll need to save the Look and then get its ID from the url (https://companyname.looker.com/looks/**[look_id]**). I'm returning the data as json, but you can return it in a variety of formats. Check the Looker API reference to see them all!

```python 
unused_content_looks = json.loads(admin_sdk.run_look(unused_content_lookid, 'json'))
```

## 3) The scary part - soft deleting content

One thing to know is that when you soft-delete content, it's not gone forever, it's just not going to be referenced by the content validator. You're moving it to a "Trash" folder and content can be restored to its original location -- as long as that original location still exists.

To soft delete, we're going to **update the 'deleted' attribute to 'true'** for each piece of content from the **unused content Look** we created.

Note that to update a look, the ID needs to be an integer. To update a dashboard, the ID needs to be a string. I recommend using if statements to ensure that you're sending the right data types to each function.

Final note here is that sometimes Looks get deleted or just can't be found anymore on the instance. I'm not sure why this is, but it causes errors. I've included some error handling here that will keep the code from stopping, while logging the missing content.

```python
for content in unused_content_results:
    content_id = content['content_usage.content_id']
    content_type = content['content_usage.content_type']
    if content_type == 'look':
        try:
            admin_sdk.update_look(int(content_id), soft_delete_body)
        except looker_sdk.error.SDKError:
            print(f'SDK Error on Look with ID {content_id}')
            continue
    if content_type == 'dashboard':
        try:
            admin_sdk.update_dashboard(content_id, soft_delete_body)
        except looker_sdk.error.SDKError:
            print(f'SDK Error on Dashboard with ID {content_id}')
```

In [2]:
'''
The following line is authorizing you to use the API on our instance, but
it requires you to have a looker.ini file with the following:

[Looker]
base_url = "https://[COMPANY_NAME].looker.com:19999"
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"
'''

admin_sdk = looker_sdk.init31()

'''
The following line will switch your user into development mode, if you'd like to make
changes directly to production, comment this line out.

Also, this will automatically put you in the branch you were last using on each project.
If you'd like to check your brach, use sdk.git_branch(project_id) where project_id is the
name of the project. Example: case_studies
'''

admin_sdk.update_session({
  "can": {},
  "workspace_id": "dev",
  "sudo_user_id": 0
})

ApiSession(can={'view': True, 'update': True}, workspace_id='dev', sudo_user_id=None)

In [52]:
# Change this to whatever your unused content Look ID is
########################### CHANGE ME ###########################
unused_content_lookid = 99999 

In [44]:
unused_content_results = json.loads(admin_sdk.run_look(unused_content_lookid, 'json'))

soft_delete_body = {
    'deleted': 'true'
}

In [ ]:
for content in unused_content_results:
    content_id = content['content_usage.content_id']
    content_type = content['content_usage.content_type']
    if content_type == 'look':
        try:
            admin_sdk.update_look(int(content_id), soft_delete_body)
        except looker_sdk.error.SDKError:
            print(f'SDK Error on Look with ID {content_id}')
            continue
    if content_type == 'dashboard':
        try:
            admin_sdk.update_dashboard(content_id, soft_delete_body)
        except looker_sdk.error.SDKError:
            print(f'SDK Error on Dashboard with ID {content_id}')